Copyright 2024 Lehi Gracia

### Save?

In [27]:
save_to_mongo = False

import subprocess, os
if save_to_mongo:
    mongod = subprocess.Popen(
        ['mongod', '--dbpath', os.path.expanduser(os.getenv('MONGODB_PATH')), '--logpath', os.path.expanduser(os.getenv('MONGODB_LOG'))]
    )

### MARKETS

In [28]:

import warnings
warnings.filterwarnings('ignore')

from yfinance_utils import list_utils, file_utils, constants
import yfinance 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA



COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

data = yfinance.download(symbols, period='1y')
gains = {}
for symbol in symbols:
    df1 = pd.DataFrame()
    df1 = data.loc[:,(slice(None),symbol)]
    df1.columns = COLUMNS
    df1['sma'] = TA.SMA(df1, period=50)
    df1 = df1[50:]
    df1.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=df1['sma'], x=df1['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()

print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol in symbols:
    print(f'{symbol:4}:{gains[names[symbol]]:6}: {names[symbol]}')



[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:
SPY : -1.53: S&P 500
QQQ :  0.98: NASDAQ 100
DIA :  -3.8: DOW JONES


### Implied Volaility

In [29]:
import yfinance
from finta import TA
from plotly.subplots import make_subplots
_t = yfinance.Ticker('^VIX')

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
_d = _t.history(period='1y')
_d['sma'] = TA.SMA(_d, period=30)

_f=make_subplots(specs=[[{"secondary_y": True}]])
_f.update_layout(title=f"Market Volatility (VIX)", title_x=0.5, width=600, height=400)
_d.reset_index(inplace=True)
_f.add_trace(
    go.Scatter(y=_d['Close'], x=_d['Date'], name="VIX"),
    secondary_y=False
)
_f.add_trace(
    go.Scatter(y=_d['sma'], x=_d['Date'], name=f'sma'),
    secondary_y=False
)
_f.show()

### Sectors

In [30]:

from yfinance_utils import list_utils
from datetime import datetime

symbols = list_utils.get_sectors()
names = list_utils.sectors
COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
NUMBER_OF_TOP_PERFORMERS_IN_MARKETS = 3

fig2=make_subplots(rows=3, cols=1)
fig2.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_MARKETS} SECTORS", title_x=0.5, width=800, height=600)

top = list_utils.get_top_performers(symbols, size=NUMBER_OF_TOP_PERFORMERS_IN_MARKETS)
_list = [i[0] for i in top]

data = yfinance.download(_list, period='1y')
counter=1
for symbol in _list:
    _d2 = pd.DataFrame()
    _d2 = data.loc[:,(slice(None),symbol)]
    _d2.columns = COLUMNS
    _d2['sma'] = TA.SMA(_d2, period=50)
    _d2.reset_index(inplace=True)
    fig2.add_trace(
        go.Scatter(y=_d2['Close'], x=_d2['Date'], name=names[symbol]),
        secondary_y=False,
        row=counter, col=1
    )
    fig2.add_trace(
        go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
        secondary_y=False,
        row=counter, col=1
    )
    counter = counter + 1
fig2.show()

today = datetime.today().strftime('%Y-%m-%d')
print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol, value in top:
    if save_to_mongo:
        file_utils.save_to_mongo({"DATE":today, "TICK":symbol, "PERFORMANCE":value, "SECTOR":names[symbol]}, "top_sectors")



[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  3 of 3 completed


PERFORMANCE IN THE LAST 30 DAYS:


###  Getting 5 best performing stocks in each of top 3 sectors

In [57]:
# Best performance stock in sectors

from yfinance_utils import list_utils
import numpy as np

NUMBER_OF_TOP_PERFORMERS_IN_SECTORS = 5
SMA_PERIOD = 50
_sym = {}
for symbol, value in top:
    _l = []
    for s in list_utils.SECTORS[symbol].keys():
        _l.append(s)
    _tp = list_utils.get_top_performers(_l, days=30, size=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS)
    _l = [i[0] for i in _tp]
    _sym[symbol] = _l
    
for i in list(_sym.keys()):
    
    _f=make_subplots(rows=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS, cols=1)
    _f.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_SECTORS} from {names[i]}", title_x=0.5, width=600, height=1000)
    
    data = yfinance.download(_sym[i], period='1y')
    counter = 1
    for symbol in _sym[i]:
        _d2 = pd.DataFrame()
        _d2 = data.loc[:,(slice(None),symbol)]
        _d2.columns = COLUMNS
        _d2['sma'] = TA.SMA(_d2, period=SMA_PERIOD)
        _d2['pct'] = _d2['sma'].pct_change()
        _d2['pct_close'] = _d2['Close'] * _d2['pct'] * 25

        _d2["Color"] = np.where(_d2["pct_close"] < 0, 'red', 'green')

        _d2.reset_index(inplace=True)
        _f.add_trace(
            go.Scatter(y=_d2['Close'], x=_d2['Date'], name=symbol,showlegend=False),
            col=1, row=counter
        )
        _f.add_trace(
            go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma {SMA_PERIOD}', showlegend=False),
            col=1, row=counter
        )
        _f.add_trace(
            go.Bar(y=_d2['pct_close'], x=_d2['Date'], name=f'change', marker_color=_d2['Color'], showlegend=False),
            col=1, row=counter
        )
        _f.update_yaxes(side='right')
        price = f"{symbol} - Date: {today}, Open:{round(_d2['Open'].iloc[-1],2)}, High:{round(_d2['High'].iloc[-1],2)}, Low:{round(_d2['Low'].iloc[-1],2)}, Close: {round(_d2['Close'].iloc[-1],2)}"
        _f.add_annotation(xref='x domain',
                        yref='y domain',
                        x=0.01,
                        y=1,
                        text=price, 
                        showarrow=False,
                        font=dict(size=10),
                        row=counter, col=1)
        counter = counter + 1
    _f.show()

if save_to_mongo:
    _sym['DATE'] = datetime.today().strftime('%Y-%m-%d')
    file_utils.save_to_mongo(_sym, "top_performers")

[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  69 of 69 completed
[*********************100%***********************]  54 of 54 completed
[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


[*********************100%***********************]  5 of 5 completed


### Top Analysts Ratings

In [22]:

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_colwidth', None)

client = MongoClient('mongodb://localhost:27017/')
db = client['bigdata']
collection = db.market

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')


res = collection.find(
                 {'DATE':today, 'script':'daily_analysts_ratings'}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2).sort_values(by="TOTAL", axis=0, ascending=False).drop_duplicates(subset=["TICK"], keep='last').iloc[:20]
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)


            TICK  UP  DOWN  HOLD    AVG  TOTAL
DATE                                          
2025-01-16  META  58     2     8  85.29     68
2025-01-16  AMZN  64     1     3  94.12     68
2025-01-16  NVDA  59     0     4  93.65     63
2025-01-16  MSFT  53     0     5  91.38     58
2025-01-16  AVGO  38     0     5  88.37     43
2025-01-16  MRVL  33     1     2  91.67     36
2025-01-16   BSX  31     0     4  88.57     35
2025-01-16   COP  25     0     3  89.29     28
2025-01-16   UNH  25     0     2  92.59     27
2025-01-16  VICI  21     0     3  87.50     24
2025-01-16   UAL  23     1     0  95.83     24
2025-01-16   HWM  21     1     2  87.50     24
2025-01-16   DAL  22     1     0  95.65     23
2025-01-16  SNPS  20     0     3  86.96     23
2025-01-16  TRGP  20     1     2  86.96     23
2025-01-16  SPGI  21     0     2  91.30     23
2025-01-16   LYV  19     2     1  86.36     22
2025-01-16   VRT  18     0     2  90.00     20
2025-01-16    GE  18     0     1  94.74     19
2025-01-16  A

### Gap up close high

In [23]:
# Gap up close high

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_gap_up_close_high']}}, 
                 {'_id':0, 'script':0, 'timestamp':0}
                 )
data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)

            TICK    HIGH   CLOSE    OPEN  CLOSE -1
DATE                                              
2025-01-16   SPY  594.35  594.17  591.64    590.33
2025-01-16   VOO  546.50  546.35  544.24    542.84
2025-01-16   HWM  123.55  123.00  122.98    122.81
2025-01-16   WMT   91.72   91.50   91.30     91.07
2025-01-16   VTI  294.66  294.23  293.49    292.71
2025-01-16    IR   91.25   90.83   90.75     90.66
2025-01-16  TSLA  423.57  423.49  413.82    409.90
2025-01-16  MSFT  429.49  428.70  424.58    419.13
2025-01-16   QQQ  519.06  519.01  513.08    513.03
2025-01-16   USD   69.68   69.59   65.69     65.52
2025-01-16   AXP  314.00  312.70  311.48    305.00
2025-01-16  ZBRA  404.40  403.00  402.72    400.77
2025-01-16  VRTX  424.58  424.58  416.96    415.13
2025-01-16   PDD  100.93  100.84  100.26     99.14
2025-01-16   PKG  237.94  237.94  235.32    235.28


### Price drops and jumps

In [24]:
# Price Ups and Downs

res = collection.find(
                 {'DATE':today, 'script':{'$in':['daily_jump_price_down','daily_jump_price_up']}}, 
                 {'_id':0, 'script':0, 'timestamp':0, 'VOLUME':0  }
                 )

data = pd.DataFrame(res).round(2)
if data.empty:
    print('No Data Today')
else:
    data.set_index('DATE', inplace=True)
    print(data)

            TICK   CLOSE  % PRICE JUMP  % VOLUME -1  % VOLUME -2
DATE                                                            
2025-01-16   SYM   32.51         23.52       760.09        15.07
2025-01-16   SMR   21.24          5.25        -9.88        36.07
2025-01-16  IONQ   36.86         12.45       -39.43        71.44
2025-01-16  AMAT  185.80          5.12        83.14        53.79
2025-01-16  LUNR   19.12          8.33       -40.32        96.95
2025-01-16  SOUN   14.15          5.52       -14.90        12.07
2025-01-16   TSM  218.89          8.10        95.87        28.93
2025-01-16   USD   69.59          6.21        -8.21        16.55
2025-01-16  QUBT   12.34         40.55        -7.21       164.18
2025-01-16  SERV   19.00          8.82       -24.24        13.23
2025-01-16  SMMT   18.18          5.27       -21.14       -21.12
2025-01-16  MNDY  241.65          5.62         7.31        28.06
2025-01-16  ENPH   63.70         -7.02        57.71       -19.05
2025-01-16   CRL  168.62 

### TTM Squeeze

In [ ]:
# TTM SQUEEZE

res = collection.find(
    {"script":"daily_ttm_squeeze", "DATE":today},
    {"_id":0, "script":0, "timestamp":0}
    )

df = pd.DataFrame(res)
df.set_index("DATE", inplace=True)
print(df)

            TICK  DAYS
DATE                  
2025-01-16    CF     6
2025-01-16   AIR     6
2025-01-16   HES     4
2025-01-16  TRGP     7
2025-01-16  KTOS    31
2025-01-16   GEV    31
2025-01-16    CF     6
2025-01-16   AIR     6
2025-01-16   HES     4
2025-01-16  TRGP     7
2025-01-16  KTOS    31
2025-01-16   GEV    31


In [26]:

print('The END')

The END
